In [2]:
from manim import *
import math
import numpy as np
from manim_slides import *
import random
import os
os.environ["PATH"] = "/Library/TeX/texbin:" + os.environ["PATH"]

/Users/mnica/py_projects/.venv/lib/python3.9/site-packages/pydub/utils.py:170: RuntimeWarning: Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work
  warn("Couldn't find ffmpeg or avconv - defaulting to ffmpeg, but may not work", RuntimeWarning)
/Users/mnica/py_projects/.venv/lib/python3.9/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(


In [3]:
DELAY = 2.0
a_color = "#FF7075" #"#ff595e" #"#90f1ef" #"#79addc" # #BLUE
b_color = "#9768de" #"#9D85C1" #6a4c93" #e9d8a6" #"#f4a261" #LIGHT_BROWN 
c_color = "#A9DE54" #"#8ac926" #"#ffee93" #"#ffef9f" #PURPLE

x_color = "#5DB4EA" #"#1982c4" #"#94d2bd" #"#2a9d8f" #BLUE_B
y_color = "#FFD35C" #"#ffca3a" #"#ffc09f" #"#ffd6e0"  #RED

bracket_color = GREY_B

t2cD = {
    r" a ": a_color,
    r" b ": b_color,
    r" c ": c_color,
    r"{3}": a_color,
    r"{4}": b_color,
    r"{5}": c_color,
    r"{12,709}" : a_color,
    r"{13,500}" : b_color,
    r"{18,541}" : c_color,
    r"{x}" : x_color,
    r"{y}" : y_color,
    r"{r}" : x_color,
    r"{s}" : y_color,
    r" ( " : bracket_color,
    r" ) " : bracket_color,
    r"\big(" : bracket_color,
    r"\big)" : bracket_color
}

In [21]:
%%manim -ql -v WARNING PythagoreanTripleGrid

FINAL_MOBJECTS = None

# === CONFIGURABLE PARAMETERS ===
x_max = 8
y_max = 4
spacing = 1.75
stroke_width = 2
color_scheme = [BLUE, GREEN, ORANGE, RED, PURPLE, TEAL]
label_buff = 0.15
tri_size = 1.25 #maximum height/width of triangle


def create_black_box(math_tex, buffer=0.1):
    """
    Creates a black rounded rectangle behind a MathTex object.

    Parameters:
    - math_tex: MathTex object to surround
    - buffer: padding around the MathTex

    Returns:
    - A VGroup containing the rectangle (ready to add behind)
    """
    rect = SurroundingRectangle(
        math_tex,
        color=BLACK,
        fill_opacity=1,
        buff=buffer,
        corner_radius=0.1
    )
    return rect

def generate_palette(n=15):
    """Generate n vivid colors spaced around the HSV color wheel."""
    seed = 2
    random.seed(seed)
    # Generate n hues randomly spaced around the color wheel
    #hues = [(0.5 - 0.1 +  0.2947*i) for i in range(n)]
    #random.shuffle(hues)

    hues = [0.4]  # Start with fixed hue

    # Keep generating hues until we have enough distinct ones
    min_dist = 0.05
    attempts = 0
    while len(hues) < n:
        h = random.random()
        # Check circular hue distance to all existing hues
        if all(min(abs(h - h2), 1 - abs(h - h2)) >= min_dist for h2 in hues):
            hues.append(h)
        attempts += 1
        if attempts > 10000:
            raise RuntimeError("Couldn't find enough distinct hues; try lowering min_dist or n.")
    

    # Shuffle all except the first one to keep 0.4 first
    #rest = hues[1:]
    #random.shuffle(rest)
    #hues[1:] = rest

    return [ManimColor.from_hsv((hues[i], 1.0, 1.0)) for i in range(n)] #


class ColorManager:
    """Keeps track of which ratio gets which color from a palette."""
    def __init__(self, palette=None):
        if palette is None:
            palette = generate_palette() #[RED, GREEN, BLUE, ORANGE, PURPLE, TEAL, GOLD, PINK, MAROON, YELLOW]
        self.palette = palette
        self.index = 0
        self.ratio_to_color = {}

    def get_color(self, ratio):
        # Round to avoid floating-point mismatch
        key = round(ratio, 4)

        # Return cached color if already assigned
        if key in self.ratio_to_color:
            return self.ratio_to_color[key]

        # Assign a new color
        color = self.palette[self.index % len(self.palette)]
        self.ratio_to_color[key] = color
        self.index += 1
        print(self.index)
        return color



# --- Scene to display grid of Pythagorean triple triangles ---
class PythagoreanTripleGrid(Scene):
    def pause(self):
        self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
    
    def construct(self):
        color_manager = ColorManager()

        # === Helper: compute triple sides ===
        def pythagorean_sides(x, y):
            a = x**2 - y**2
            b = 2 * x * y
            c = x**2 + y**2
            return a, b, c


        # === Helper: create triangle with centered edge labels ===
        def make_triangle(x, y):
            a, b, c = pythagorean_sides(x, y)

            # Construct a right triangle manually (not a RegularPolygon)
            p1 = ORIGIN
            p2 = LEFT * a
            p3 = UP * b
            ratio = max(a / b, b / a)
            #mycolor = color_manager.get_color(ratio)
            mycolor = WHITE
            tri = Polygon(p1, p2, p3, color= mycolor, stroke_width=stroke_width)
            if a > b:
                tri.set_width(tri_size)
            else:
                tri.set_height(tri_size)

            

            # Compute centers of edges (like get_center_of_edges)
            vertices = tri.get_vertices()
            centers = []
            for i in range(len(vertices)):
                p1, p2 = vertices[i], vertices[(i + 1) % len(vertices)]
                guide_line = Line(p1, p2)
                center = guide_line.get_center()
                normal_direction = guide_line.copy()
                normal_direction.rotate(+PI / 2)
                vector_normal_direction = normal_direction.get_unit_vector()
                direction = Dot(center).shift(vector_normal_direction * label_buff).get_center()
                centers.append(direction)

            # Create labels centered on each edge
            labels_text = [f"{a}", f"{c}", f"{b}"]
            my_colors = [a_color, c_color, b_color]
            labels = VGroup(*[MathTex(lbl,color=the_color).scale(0.5).move_to(pos)
                              for lbl, pos, the_color in zip(labels_text, centers, my_colors)])

            return VGroup(tri, labels)


        #self.next_section(skip_animations=True)

        
        # === Create grid of triangles (x > y) ===
        triangles = VGroup()
        for xi in range(2, x_max + 1):
            for yi in range(1, min(x_max, xi - 1) + 1):
                tri = make_triangle(xi, yi)
                x_offset = (xi - 2) * spacing
                y_offset = -(yi - 1) * spacing
                tri.move_to(RIGHT * x_offset + UP * y_offset)
                triangles.add(tri)

        triangles.to_corner(UL).shift(0.5*DOWN + 0.4*RIGHT)

        triangles[0].generate_target()
        triangles[0].set_x(0)
        triangles[0].set_y(0)
        triangles[0].set_width(2.0)


        LLE_logo = ImageMobject("LLE_logo.png").scale(0.7)


        tex_scale = 1.2
        full_header = MathTex(r" a ",r"^2","+",r" b ",r"^2","=",r" c ",r"^2",r"\text{ has infinitely many integer solutions!}",tex_to_color_map=t2cD).scale(tex_scale)
        full_header_copy = full_header.copy()
        
        inf_solutions = full_header[-1]
        abc = full_header[0:-1]

        eqn = MathTex(r"{3}",r"^2","+",r"{4}",r"^2","=",r"{5}",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        print(*enumerate(eqn),sep="\n")
        eqn.next_to(triangles[0],DOWN)
        LLE_logo.next_to(eqn,DOWN)
        Group(abc,LLE_logo,eqn,triangles[0]).arrange(DOWN).set_y(0)
        self.add(LLE_logo)
        self.pause()


        self.play(Write(eqn),run_time=4.0)
        self.wait(6.0)
        self.play(LaggedStart(Create(triangles[0][0]),
                              ReplacementTransform(eqn[0].copy(),triangles[0][1][0]),
                              ReplacementTransform(eqn[3].copy(),triangles[0][1][2]),
                              ReplacementTransform(eqn[6].copy(),triangles[0][1][1]),
                              lag_ratio=0.5
        ),run_time=4.0)
        self.wait(7.0)
        self.play(Write(abc),run_time=3.0)
        self.wait(4.0)

        full_header_copy.next_to(LLE_logo,UP)

        self.play(LaggedStart( ReplacementTransform(abc,full_header_copy[0:-1]), Write(full_header_copy[-1]), lag_ratio=0.8)  )
        #VGroup(abc.target,inf_solutions).arrange(RIGHT).next_to(LLE_logo,UP).set_x(0)
        #abc.target.align_to(inf_solutions[0],DOWN)
        #self.play(LaggedStart(MoveToTarget(abc),Write(inf_solutions),lag_ratio=0.8))


        
        self.wait(15.0)
        self.pause()

        ldots_mob = MathTex(r"\ldots",tex_to_color_map=t2cD).scale(tex_scale)
        ldots = VGroup()
        for yi in range(x_max,0,-1):
            ldots_mob.next_to(triangles[-yi])
            ldots.add(ldots_mob.copy())


        print(*enumerate(ldots),sep="\n")
        
        self.play(LaggedStart(MoveToTarget(triangles[0]), 
                  *[FadeIn(triangles[i],shift=DOWN) for i in range(1,len(triangles))],
                  *[FadeIn(ldots[i],shift=DOWN) for i in range(len(ldots))],
                  lag_ratio=0.2),
                 FadeOut(LLE_logo,eqn))

        
        self.next_section()
        self.wait(2)

        self.pause()



        #self.next_section()
        P322 = ImageMobject("Plimpton_322.png").scale(0.7).shift(0.6*UP)
        P322_label = txt = Text(
            "Plimpton 322 Clay Tablet",
            font="Arial Black",
            color=WHITE,
            stroke_width=2,
            stroke_color=BLACK,
        )
        P322_label.width = P322.width*0.75
        P322_label.next_to(P322,DOWN)
        P322_label.align_to(P322,UP)
        P322.shift(0.15*UP)
        self.play(FadeIn(P322,P322_label))
        
        Edgar = ImageMobject("Edgar_James_Banks.jpg").scale(1.3).next_to(P322,RIGHT) #.shift(0.8*UP)
        edgar_label = txt = Text(
            "Edgar J. Banks",
            font="Arial Black",
            color=WHITE,
            stroke_width=1,
            stroke_color=BLACK,
        )#MathTex(r"\text{Edgar J. Banks}",color=GRAY)
        edgar_label.width = Edgar.width
        edgar_label.next_to(Edgar,DOWN)
        edgar_label.align_to(Edgar,DOWN)
        self.play(FadeIn(Edgar,edgar_label,shift=RIGHT))
        self.pause()

        

        eqnBabylonian = MathTex(r"{12,709}",r"^2","+",r"{13,500}",r"^2","=",r"{18,541}",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqnBabylonian.next_to(P322,DOWN,buff=0.1)
        black_box_eqn = create_black_box(eqnBabylonian,buffer=0.2)
        eqnBabylonian.z_index = black_box_eqn.z_index+1    
        eqnBabylonian = VGroup(eqnBabylonian,black_box_eqn)
        self.play(GrowFromEdge(eqnBabylonian,UP))
        self.pause()

        triangles.set_z_index(-100)
        self.play(*[FadeOut(mob,shift=DOWN) for mob in [triangles, ldots, P322,P322_label,Edgar,edgar_label,eqnBabylonian]])
        
        #,FadeOut(triangles,shift=DOWN))

        

        

        global FINAL_MOBJECTS
        FINAL_MOBJECTS = VGroup(full_header_copy, triangles)
        self.pause()
        
        #self.add(triangles)
        #triangles.move_to(ORIGIN)
        #self.play(LaggedStartMap(FadeIn, triangles, lag_ratio=0.05))
        #self.wait()



        # Note: The triple 30, 72, 78 is missing (generated by setting xy to be a perfect square but x,y themselves are not perfect squares, x=2^3*3 and y=2*3^3)


        #Podcast Quote:So 3 is the power of 2 plus force of power 2 equals 5 to the power of 2. 9 plus 16 equals 25. 
        # #That makes sense. Makes sense. 
        # All whole numbers. This is actually a Pythagorean triple. 
        # So, you know, in a right-handed triangle, the square of the hypotenuse, the side of the triangle. 
        # A square plus B squared equals C squared. Opposite the triangle is equal to the sum of the squares and the two-squared. 
        # A squared plus B squared equals C squared, as you say, Tom. 
        # This is Pythagoras' theorem, right? And in fact, there are an infinite number of positive integer solutions for this equation. 
        # A, B and C can be an infinite number of positive integers. Whoa. Okay. 
        # This is, this has been well known since like Babylonian times. Right. And we both knew that, right, Alan? Since Babylonian times, I knew that.
        #Starts at 10:20

Manim Community v0.19.0

<string>:123: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().
<string>:121: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().
<string>:167: DeprecationWarning: This method is not guaranteed to stay around. Please prefer setting the attribute normally or with Mobject.set().


(0, SingleStringMathTex('{3}'))
(1, SingleStringMathTex('^2'))
(2, SingleStringMathTex('+'))
(3, SingleStringMathTex('{4}'))
(4, SingleStringMathTex('^2'))
(5, SingleStringMathTex('='))
(6, SingleStringMathTex('{5}'))
(7, SingleStringMathTex('^2'))


(0, MathTex('\\ldots'))
(1, MathTex('\\ldots'))
(2, MathTex('\\ldots'))
(3, MathTex('\\ldots'))
(4, MathTex('\\ldots'))
(5, MathTex('\\ldots'))
(6, MathTex('\\ldots'))
(7, MathTex('\\ldots'))


In [22]:
%%manim -ql -v WARNING PythagoreanTripleAlgebra

def crosshatch_rectangle(rect, spacing=0.15, angle=90*DEGREES, color=WHITE,lwidth=1):
    """
    Create crosshatch lines clipped to a Rectangle or Square,
    without using boolean operations.
    """

    # Rectangle corner coordinates (in world space)
    my_UL = rect.get_corner(UL)
    my_UR = rect.get_corner(UR)
    my_DL = rect.get_corner(DL)
    my_DR = rect.get_corner(DR)

    # Convert rectangle boundary to a list of segment endpoints
    edges = [
        (my_UL, my_UR),  # top
        (my_UR, my_DR),  # right
        (my_DL, my_DR),  # bottom
        (my_DL, my_UL),  # left
    ]

    # Determine a sufficiently large diagonal span
    width = rect.width
    height = rect.height
    diag = width + height

    # Number of diagonals needed
    n = int(diag / spacing) + 4

    hatch = VGroup()

    # Create and clip lines
    for i in range(-n, n+1):
        # Base line (long)
        p1 = LEFT*diag
        p2 = RIGHT*diag 

        # Rotate & shift
        R = rotation_matrix(angle, OUT)[:3,:3]
        p1 = R @ p1
        p2 = R @ p2

        # Shift the diagonal line by i steps upward
        shift_vec = (i +0.5) * spacing * (R@UP)
        p1 = p1 + shift_vec
        p2 = p2 + shift_vec

        p1 += rect.get_center()
        p2 += rect.get_center()

        # ------------------------------------------------------------------
        # Compute intersections between this diagonal line and the rectangle
        # ------------------------------------------------------------------
        pts = []
        for (a, b) in edges:
            inter = line_intersection(p1, p2, a, b)
            if inter is not None:
                # Check if intersection lies within rectangle edge segment
                if point_on_segment(inter, a, b):
                    # Check if intersection lies on (infinite) diagonal line segment
                    pts.append(inter)

        # We keep segments only if we have exactly 2 intersection points
        if len(pts) == 2:
            seg = Line(pts[0], pts[1], color=color, stroke_width=lwidth)
            seg.set_fill(color=color, opacity=0.1)
            hatch.add(seg)

    return hatch


# -------------------------------------------------------
# Helper: line intersection
# -------------------------------------------------------
def line_intersection(p1, p2, p3, p4):
    """
    Intersection of line segment p1→p2 with line segment p3→p4.
    Returns None or a numpy array.
    """

    x1, y1, _ = p1
    x2, y2, _ = p2
    x3, y3, _ = p3
    x4, y4, _ = p4

    denom = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    if abs(denom) < 1e-8:
        return None  # parallel lines

    px = ((x1*y2 - y1*x2)*(x3 - x4) - (x1-x2)*(x3*y4 - y3*x4)) / denom
    py = ((x1*y2 - y1*x2)*(y3 - y4) - (y1-y2)*(x3*y4 - y3*x4)) / denom
    return np.array([px, py, 0])


# -------------------------------------------------------
# Helper: check if point lies on the interior of segment AB
# -------------------------------------------------------
def point_on_segment(p, a, b):
    return (
        np.allclose(np.linalg.norm(np.cross(b-a, p-a)), 0, atol=1e-6)
        and np.dot(p-a, p-b) <= 0
    )


def create_black_box(math_tex, buffer=0.1):
    """
    Creates a black rounded rectangle behind a MathTex object.

    Parameters:
    - math_tex: MathTex object to surround
    - buffer: padding around the MathTex

    Returns:
    - A VGroup containing the rectangle (ready to add behind)
    """
    rect = SurroundingRectangle(
        math_tex,
        color=BLACK,
        fill_opacity=1,
        buff=buffer,
        corner_radius=0.1
    )
    return rect

# --- Scene to display grid of Pythagorean triple triangles ---
class PythagoreanTripleAlgebra(Scene):
    def pause(self):
        self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
    
    def construct(self):
        full_header, triangles = FINAL_MOBJECTS.copy()
        self.add(full_header)
        #self.pause()
        #self.next_section(skip_animations=True)
        tex_scale = 1.25
        global t2cD
        print(t2cD)
        eqn1 = MathTex(r"\big(",r"{x}",r"+",r"{y}",r"\big)",r"^2", r"=", r"{x}^2", r"+", r"2", r"{x}{y}", r" + ", r"{y}^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqn2 = MathTex(r"\big(",r"{x}",r"-",r"{y}",r"\big)",r"^2", r"=", r"{x}^2", r"-", r"2",r"{x}{y}", r" + ", r"{y}^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqn3 = MathTex(r"4{x}{y}", r"=", r"\big(",r"{x}",r"+",r"{y}",r"\big)",r"^2",r"-",r"\big(",r"{x}",r"-",r"{y}",r"\big)", r"^2", tex_to_color_map=t2cD).scale(tex_scale)
        eqn3a = MathTex(r"\big(",r"{x}",r"-",r"{y}",r"\big)",r"^2",r"+",r"4{x}{y}", r"=", r"\big(",r"{x}",r"+",r"{y}",r"\big)",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqn3b = MathTex(r"\big(",r"{r}",r" ^2",r"-",r"{s}",r" ^2",r"\big)",r"^2",r"+",r"4",r"{r}",r"{^2}",r"{s}",r"{^2}", r"=", r"\big(",r"{r} ^2",r"+",r"{s} ^2",r"\big)",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqn3c = MathTex(r"\big(",r"{r}",r" ^2",r"-",r"{s}",r" ^2",r"\big)",r"^2",r"\!",r"+",r"\!",r" ( ", r"2",r"{r}",r"{s}", r" ) ",r"{^2}", r"=", r"\big(",r"{r} ^2", r"+",r"{s}",r" ^2",r"\big)",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        

        equations = VGroup(*[eqn1, eqn2, eqn3, eqn3a, eqn3b, eqn3c]).arrange(DOWN,buff=0.25)
        eqn3b.align_to(eqn3, UP)
        eqn3c.align_to(eqn3, UP)
        eqn3a.align_to(eqn3, UP)
        equations.shift(DOWN*0.8)

        
        
        # Function to find the submobject corresponding to "="
        def get_equals_part(eq):
            for part in eq:
                # The latex source string for that submobject
                if part.get_tex_string() == "=":
                    return part
            raise ValueError("No '=' found in this MathTex object.")

        
        def get_index_list(mathtexmob, part):
            answers = []
            for ix in range(len(mathtexmob)): 
                # The latex source string for that submobject
                if mathtexmob[ix].get_tex_string() == part:
                    answers.append(ix)
            if len(answers) > 0:
                return answers
            else:
                raise ValueError(f"No {part} found in this MathTex object.")

        # Step 1: Find x positions of "=" in each equation
        equals_x_positions = [get_equals_part(eq).get_center()[0] for eq in equations]

        # Step 2: Choose alignment position (e.g. max so nothing goes off screen)
        target_x = max(equals_x_positions)

        # Step 3: Shift each equation so that their "=" aligns
        for eq, eq_x in zip(equations, equals_x_positions):
            eq.shift((target_x - eq_x) * RIGHT)

        # Step 4: Arrange vertically
        #VGroup(*equations).arrange(DOWN, aligned_edge=LEFT)
        equations.to_edge(RIGHT,buff=0.8)

        eqn_down_shift = 0.25
        eqn3.shift(DOWN* eqn_down_shift)
        for mob in [eqn3a,eqn3b,eqn3c]:
            mob.shift(RIGHT*2.8 + DOWN* eqn_down_shift)

        plugin = MathTex(r"\text{Plug in:}").scale(tex_scale*0.8)
        x_is_u2 = MathTex(r"{x}", r"\!=\!", r"{r}^2",tex_to_color_map=t2cD).scale(tex_scale)
        #comma = MathTex(r",",tex_to_color_map=t2cD).scale(tex_scale)
        y_is_v2 = MathTex(r"{y}", r"\!=\!", r"{s}^2",tex_to_color_map=t2cD).scale(tex_scale)
        xyuv = VGroup(plugin,x_is_u2,y_is_v2).arrange(RIGHT,buff=0.4,aligned_edge=DOWN).to_edge(DOWN,buff=0.8) #.next_to(full_header,DOWN,buff=0.4).to_edge(LEFT)

        # Step 1: Find x positions of "=" in each equation
        xyuv_0_ys = [mob[min(1, len(mob)-1)].get_center()[1] for mob in xyuv]

        # Step 2: Choose alignment position (e.g. max so nothing goes off screen)
        target_y = max(xyuv_0_ys)

        # Step 3: Shift each equation so that their "=" aligns
        for mob,mob_y in zip(xyuv, xyuv_0_ys):
            mob.shift((target_y - mob_y) * UP)

        brace_color = GREY_C
        u_ix = get_index_list(eqn3c, r"{r}")[0]
        v_ix = get_index_list(eqn3c, r"{s}")[0]
        a_brace = Brace(eqn3c[u_ix:v_ix+2], DOWN, color=brace_color, buff=0.4)
        a = MathTex(r"{ a }",tex_to_color_map=t2cD).scale(tex_scale)
        a_brace.put_at_tip(a)


        u_ix = get_index_list(eqn3c, r"{r}")[1]
        v_ix = get_index_list(eqn3c, r"{s}")[1]
        b_brace = Brace(eqn3c[u_ix-1:v_ix+1], DOWN, color=brace_color, buff=0.4)
        b = MathTex(r"{ b }",tex_to_color_map=t2cD).scale(tex_scale)
        b_brace.put_at_tip(b)
        

        u_ix = get_index_list(eqn3c, r"{r}")[2]
        v_ix = get_index_list(eqn3c, r"{s}")[2]
        c_brace = Brace(eqn3c[u_ix:v_ix+2], DOWN, color=brace_color, buff=0.4)
        c = MathTex(r"{ c }",tex_to_color_map=t2cD).scale(tex_scale)
        c_brace.put_at_tip(c)

        b.align_to(a,DOWN)
        
        #b_brace = Brace(eqn3c[7:11], UP, buff=0.1)
        #c_brace = Brace(eqn3c[11:15], UP, buff=0.1)
        #brace_group = VGroup(a_brace, b_brace, c_brace).to_edge(LEFT)

        #self.next_section(skip_animations = True)
        print(*enumerate(eqn1),sep="\n")
        print(get_index_list(eqn1,"="))

        

        def binomial_theorem_anims(eqn, pauses=True):
            square_ixs = get_index_list(eqn,"^2") 
            x_ixs = get_index_list(eqn,"{x}")
            self.play(ReplacementTransform(eqn[x_ixs[0]].copy(),eqn[square_ixs[1]-1],path_arc=-PI/2),
                    ReplacementTransform(eqn[square_ixs[0]].copy(),eqn[square_ixs[1]],path_arc=-PI/2))
            if pauses:
                self.pause()

            y_ixs = get_index_list(eqn,"{y}")
            self.play(ReplacementTransform(eqn[y_ixs[0]].copy(),eqn[square_ixs[2]-1:square_ixs[2]+1],path_arc=-PI/2),
                    ReplacementTransform(eqn[y_ixs[0]-1].copy(),eqn[square_ixs[2]-2],path_arc=-PI/2), #animate +
                    ReplacementTransform(eqn[square_ixs[0]].copy(),eqn[square_ixs[2]],path_arc=-PI/2))
            if pauses:
                self.pause()

            two_ix = get_index_list(eqn,"2")[0]
            self.play(ReplacementTransform(eqn[x_ixs[0]].copy(),eqn[x_ixs[2]],path_arc=-PI/2),
                    ReplacementTransform(eqn[y_ixs[0]].copy(),eqn[y_ixs[1]],path_arc=-PI/2),
                    ReplacementTransform(eqn[square_ixs[0]].copy(),eqn[two_ix],path_arc=-PI/2),
                    ReplacementTransform(eqn[y_ixs[0]-1].copy(),eqn[two_ix-1],path_arc=-PI/2))
            if pauses:
                self.pause()

        eqn1_eq_ix = get_index_list(eqn1,"=")[0]
        slow_run_time = 3.0
        self.play(Write(eqn1[0:eqn1_eq_ix]),run_time=slow_run_time)
        self.pause()
        self.play(Write(eqn1[eqn1_eq_ix])) #animate =
        binomial_theorem_anims(eqn1) #(x+y)^2 expanded

        #####
        ##### square animations
        #####

        # lengths (manim units)
        x = 2.2
        y = 1.32

        # Outer big square
        big_square = Square(side_length=x+y, color=WHITE)
        square_buff = 0.8
        big_square.next_to( full_header, DOWN, buff=square_buff).to_edge(LEFT,buff=square_buff)
        #big_square.next_to(eqn2[0:eqn1_eq_ix],DOWN,buff=0.7).shift(0.5*LEFT)

        # Regions
        # Top-left x^2
        tl = Rectangle(width=x, height=x).move_to(
            big_square.get_corner(UL) + DOWN*x/2 + RIGHT*x/2
            
        )
        #tl.set_fill(color=x_color, opacity=0.3)
        # Top-right xy
        tr = Rectangle(width=y, height=x).move_to(
            big_square.get_corner(UR) + DOWN*x/2 + LEFT*y/2
        )

        # Bottom-left xy
        bl = Rectangle(width=x, height=y).move_to(
            big_square.get_corner(DL) + UP*y/2 + RIGHT*x/2
        )

        # Bottom-right y^2
        br = Rectangle(width=y, height=y).move_to(
            big_square.get_corner(DR) + UP*y/2 + LEFT*y/2
        )

        # Labels for areas
        label_x2 = MathTex("{x}^2",tex_to_color_map=t2cD).move_to(tl.get_center())
        label_xy1 = MathTex("{x}{y}",tex_to_color_map=t2cD).move_to(tr.get_center())
        label_xy2 = MathTex("{x}{y}",tex_to_color_map=t2cD).move_to(bl.get_center())
        label_y2 = MathTex("{y}^2",tex_to_color_map=t2cD).move_to(br.get_center())

        # Side labels
        label_x_top = MathTex("{x}",tex_to_color_map=t2cD).next_to(
            big_square.get_top(), UP, buff=0.15
        ).shift(LEFT * y/2)

        label_y_top = MathTex("{y}",tex_to_color_map=t2cD).next_to(
            big_square.get_top(), UP, buff=0.15
        ).shift(RIGHT * x/2)

        label_plus_top = MathTex("+",tex_to_color_map=t2cD)
        midpoint = (label_x_top.get_center()*y + label_y_top.get_center()*x) / (x+y)
        label_plus_top.move_to(midpoint)

        label_x_left = MathTex("{x}",tex_to_color_map=t2cD).next_to(
            big_square.get_left(), LEFT, buff=0.15
        ).shift(UP * y/2)

        label_y_left = MathTex("{y}",tex_to_color_map=t2cD).next_to(
            big_square.get_left(), LEFT, buff=0.15
        ).shift(DOWN * x/2)


        label_plus_left = MathTex("+",tex_to_color_map=t2cD)
        midpoint = (label_x_left.get_center()*y + label_y_left.get_center()*x) / (x+y)
        label_plus_left.move_to(midpoint)

        x_ixs = get_index_list(eqn1,"{x}")
        plus_ixs = get_index_list(eqn1,"+")
        y_ixs = get_index_list(eqn1,"{y}")


        self.play(Create(big_square))
        self.play(ReplacementTransform(eqn1[x_ixs[0]].copy(),label_x_top),
                  ReplacementTransform(eqn1[y_ixs[0]].copy(),label_y_top),
                  ReplacementTransform(eqn1[plus_ixs[0]].copy(),label_plus_top))
        self.play(ReplacementTransform(eqn1[x_ixs[0]].copy(),label_x_left),
                  ReplacementTransform(eqn1[y_ixs[0]].copy(),label_y_left),
                  ReplacementTransform(eqn1[plus_ixs[0]].copy(),label_plus_left))
        self.play(*[Create(mob) for mob in [tl,tr,bl,br]])
        self.pause()



        #self.add(big_square,tl,tr,bl,br,label_x_top,label_y_top,label_x_left,label_y_left)

        x_spacing = 0.22
        y_spacing = 0.11
        parts = [(tl,label_x2,"x","x"),(br,label_y2,"y","y"),(bl,label_xy2,"x","y"),(tr,label_xy1,"y","x")]
        hatches_and_labels = VGroup()
        for rect, label, vert_type, horz_type in parts:
            vert_color = x_color if vert_type == "x" else y_color
            horz_color = x_color if horz_type == "x" else y_color
            vert_s = x_spacing if vert_type == "x" else y_spacing
            horz_s = x_spacing if horz_type == "x" else y_spacing

            vert_lwidth = 3 if vert_type == "x" else 1
            horz_lwidth = 3 if horz_type == "x" else 1
            
            v_hatch = crosshatch_rectangle(rect, angle= 75*DEGREES, color=vert_color,lwidth=vert_lwidth,spacing=vert_s)
            h_hatch = crosshatch_rectangle(rect, angle=-15*DEGREES, color=horz_color,lwidth=horz_lwidth,spacing=horz_s)
            black_box = create_black_box(label)
            label.z_index = black_box.z_index+1
            self.play(GrowFromEdge(v_hatch,edge=UP))
            self.play(GrowFromEdge(h_hatch,edge=LEFT))
            self.play(GrowFromCenter(VGroup(black_box,label)))
            hatches_and_labels += VGroup(v_hatch,h_hatch,black_box,label)
            self.pause()




        eqn2_eq_ix = get_index_list(eqn2,"=")[0]
        self.play(Write(eqn2[0:eqn2_eq_ix+1]))
        binomial_theorem_anims(eqn2,pauses=False)
        self.pause()


        self.next_section()

        #fadeout the squqrew
        #self.play(FadeOut(VGroup(hatches_and_labels,big_square,tl,tr,bl,br,label_x_top,label_y_top,label_x_left,label_y_left)))

        eqn1_sq_ix = get_index_list(eqn1,"^2")[0]
        eqn3_sq_ix = get_index_list(eqn3,"^2")[0] #first square is the one from (x+y)^2
       
        
        self.play(ReplacementTransform(eqn1[0:eqn1_sq_ix+1].copy(),eqn3[eqn3_sq_ix-5:eqn3_sq_ix+1],path_arc=-PI/2))


    
        
        eqn2_sq_ix = get_index_list(eqn2,"^2")[0]
        eqn3_sq_ix = get_index_list(eqn3,"^2")[1] #second square is the one from (x-y)^2
        eqn3_minux_ix = get_index_list(eqn3,"-")[0] 
        
        self.play(ReplacementTransform(eqn2[0:eqn2_sq_ix+1].copy(),eqn3[eqn3_sq_ix-5:eqn3_sq_ix+1],path_arc=-PI/2),
                  FadeIn(eqn3[eqn3_minux_ix],shift=RIGHT)) # copy over (x-y)^2

        self.pause()


        eqn3_4_ix = get_index_list(eqn3,"4")[0]
        eqn1_2_ix = get_index_list(eqn1,"2")[0]
        eqn2_2_ix = get_index_list(eqn2,"2")[0]
        eqn3_eq_ix = get_index_list(eqn3,"=")[0]
        self.play(ReplacementTransform(VGroup(eqn2[eqn2_2_ix:eqn2_2_ix+3],eqn1[eqn1_2_ix:eqn1_2_ix+3]).copy(),eqn3[eqn3_4_ix:eqn3_4_ix+3],path_arc=-PI/2),
                  FadeIn(eqn3[eqn3_eq_ix],shift=LEFT))
        self.pause()


        

        print("eqn3:")
        print(*enumerate(eqn3))
        print("eqn3a:")
        print(*enumerate(eqn3a))




        self.play(
            # (x+y)^2 moves to the right side in eqn3a
            ReplacementTransform(eqn3[4:10], eqn3a[11:17]),
            # 4xy moves to the middle
            ReplacementTransform(eqn3[0:3],  eqn3a[7:10]),
            # (x-y)^2 moves to the left
            ReplacementTransform(eqn3[11:17], eqn3a[0:6],path_arc=-PI/2),
            # equals sign lines up
            ReplacementTransform(eqn3[3], eqn3a[10]),
            # *** minus between squares becomes plus ***
            ReplacementTransform(eqn3[10],     eqn3a[6],path_arc=-PI/2),
        )
        self.pause()

        eqn3 = eqn3a #replacement since future animations refer to eqn3 not eqn3a



        #self.play(FadeOut(VGroup(hatches_and_labels,big_square,tl,tr,bl,br,label_x_top,label_y_top,label_x_left,label_y_left)))
        self.play(FadeOut(VGroup(hatches_and_labels[0:2],tl,br)))

        remaining_square = VGroup(big_square,bl,tr,label_x_top,label_y_top,label_x_left,label_y_left,label_plus_top,label_plus_left)


        wide_rect = VGroup(hatches_and_labels[2],bl)
        tall_rect = VGroup(hatches_and_labels[3],tr)
        remaining_square += wide_rect
        remaining_square += tall_rect

        wide_rect.generate_target()
        wide_rect.target.move_to(big_square.get_corner(DR) + UP*y/2 + LEFT*x/2)

        wide_rect_copy = wide_rect.copy()
        tall_rect_copy = tall_rect.copy()
        self.play(MoveToTarget(wide_rect)) 
        self.wait(0.25)
        
        wide_rect_copy.move_to(big_square.get_corner(UL) + DOWN*y/2 + RIGHT*x/2)
        tall_rect_copy.move_to(big_square.get_corner(DL) + UP*x/2 + RIGHT*y/2)
        self.play(ReplacementTransform(tall_rect.copy(),tall_rect_copy,path_arc = PI/2))
        remaining_square += tall_rect_copy
        self.play(ReplacementTransform(wide_rect.copy(),wide_rect_copy,path_arc = -PI/2))
        remaining_square += wide_rect_copy
        self.wait(0.25)
        
        x_minus_y_2 = MathTex(r"( {x} - {y} )^2",tex_to_color_map=t2cD).move_to(big_square.get_center())
        x_minus_y_2.width = (x - y)
        x_minus_y_2.rotate(PI/4)
        remaining_square += x_minus_y_2
        self.play(FadeIn(x_minus_y_2))
        #tr_copy = VGroup(tr.copy(),hatches_and_labels[2].copy())
        #tr_copy.move_to(big_square.get_corner(DL) + RIGHT*x/2 + UP*y/2)

        #bl_new = VGroup(bl,hatches_and_labels[3])
        #bl_new.generate_target()
        #bl_new.target.move_to(big_square.get_corner(DR) + UP*y/2 + LEFT*x/2)

        #bl_copy = VGroup(bl.copy(),hatches_and_labels[3].copy())
        #bl_copy.move_to(big_square.get_corner(UL) + DOWN*x/2 + RIGHT*y/2)

        self.pause()

        
        

        

        self.play(Write(xyuv))
        self.pause()


        key_map_dict = {r"{x}":r"{r}",r"{y}":r"{s}"}
        xyuv_copy = xyuv.copy()
        xyuv_copy[1].generate_target()
        xyuv_copy[1].target.move_to(eqn3b.get_center()-1.0*RIGHT)
        xyuv_copy[1].target.set_opacity(0.0)
        xyuv_copy[2].generate_target()
        xyuv_copy[2].target.move_to(eqn3b.get_center()-1.0*LEFT)
        xyuv_copy[2].target.set_opacity(0.0)
        self.play(MoveToTarget(xyuv_copy[1]), #path_arc=-PI/2),
                  MoveToTarget(xyuv_copy[2]), #path_arc=-PI/2),
                  TransformMatchingTex(eqn3,eqn3b,key_map=key_map_dict,shift=UP),
                  run_time=slow_run_time)
        self.pause()


        key_map_dict = {r"4":r"2"}
        self.play(TransformMatchingTex(eqn3b,eqn3c,key_map=key_map_dict,shift=DOWN))
        self.pause()


        self.play(LaggedStart(Create(a_brace),Write(a),Create(b_brace),Write(b),Create(c_brace),Write(c),lag_ratio=0.5))
        self.pause()

        final_soln = MathTex(r"\text{ solved by }", r" a \!=\! {r}^2 \!-\! {s}^2,", r"\; b \!=\! 2{r}{s}", r",\; c \!=\! {r}^2 \!+\! {s}^2",tex_to_color_map=t2cD).scale(1.15)
        print(*enumerate(full_header),sep="\n")
        final_soln.align_to(full_header[-1],LEFT)
        final_soln.align_to(full_header[-1],DOWN)
        final_soln.shift(0.04*UP)


        a_ix = get_index_list(final_soln,r" a ")[0]
        b_ix = get_index_list(final_soln,r" b ")[0]
        c_ix = get_index_list(final_soln,r" c ")[0]
        self.play(LaggedStart(FadeOut(full_header[-1],shift=UP),
                              Write(final_soln),
                              ReplacementTransform(a.copy(),final_soln[a_ix]),
                              ReplacementTransform(b.copy(),final_soln[b_ix]),
                              ReplacementTransform(c.copy(),final_soln[c_ix]),
                              lag_ratio=0.2))
        self.pause()


        triangles.shift(0.35*DOWN + 0.2*LEFT)

        self.next_section()
        self.play(FadeIn(triangles), FadeOut(remaining_square,a_brace,b_brace,c_brace,a,b,c,eqn1,eqn2,eqn3c,xyuv))
        self.pause()    

    
        label_scale = 0.8
        u_labels = VGroup()
        for xi in range(x_max-1):
            label = MathTex(" {r} \!=\! "+str(xi+2),tex_to_color_map=t2cD).scale(label_scale)
            label.next_to(triangles[xi*(xi+1)//2],UP,buff=0.1)
            if xi > 0:
                label.align_to(u_labels[0],DOWN)
            u_labels.add(label)
        
        



        v_labels = VGroup()
        n_v_labels = x_max
        for yi in range(n_v_labels):
            label = MathTex(" {s} \!=\! "+str(n_v_labels - yi - 1),tex_to_color_map=t2cD).scale(label_scale)
            label.next_to(triangles[-yi-1],RIGHT,buff=0.5)
            if yi > 0:
                label.align_to(v_labels[0],LEFT)
            v_labels.add(label)

        
        self.play(LaggedStart(*[FadeIn(mob,shift=RIGHT) for mob in u_labels],*reversed([FadeIn(mob,shift=DOWN) for mob in v_labels]),lag_ratio=0.2))
        self.pause()

         #self.next_section()
        P322 = ImageMobject("Plimpton_322.png").scale(0.7).shift(0.97*UP)
        self.play(FadeIn(P322))
        self.pause()

        example = MathTex(r"\text{Example: }","{r} \!=\! 125", ",", "{s} \!=\! 54",tex_to_color_map=t2cD).scale(tex_scale)
        example_2 = MathTex(r"( {r}-{s} )^2 + ( 2{r}{s})^2 = ( {r}+{s} )^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqn_final = MathTex(r"{12,709}",r"^2","+",r"{13,500}",r"^2","=",r"{18,541}",r"^2",tex_to_color_map=t2cD).scale(tex_scale)
        eqnBabylonian = VGroup(example,example_2,eqn_final).arrange(DOWN,buff=0.1).scale(0.7)
        eqnBabylonian.next_to(P322,DOWN,buff=0.1)
        black_box_eqn = create_black_box(eqnBabylonian,buffer=0.2)
        eqnBabylonian.z_index = black_box_eqn.z_index+1    
        eqnBabylonian = VGroup(eqnBabylonian,black_box_eqn)
        self.play(FadeIn(black_box_eqn),GrowFromEdge(example,UP))
        self.pause()

        self.play(GrowFromEdge(example_2,UP))
        self.play(GrowFromEdge(eqn_final,UP))
        self.pause()



        


Manim Community v0.19.0

{' a ': '#FF7075', ' b ': '#9768de', ' c ': '#A9DE54', '{3}': '#FF7075', '{4}': '#9768de', '{5}': '#A9DE54', '{12,709}': '#FF7075', '{13,500}': '#9768de', '{18,541}': '#A9DE54', '{x}': '#5DB4EA', '{y}': '#FFD35C', '{r}': '#5DB4EA', '{s}': '#FFD35C', ' ( ': ManimColor('#BBBBBB'), ' ) ': ManimColor('#BBBBBB'), '\\big(': ManimColor('#BBBBBB'), '\\big)': ManimColor('#BBBBBB')}
(0, SingleStringMathTex('\\big('))
(1, SingleStringMathTex('{x}'))
(2, SingleStringMathTex('+'))
(3, SingleStringMathTex('{y}'))
(4, SingleStringMathTex('\\big)'))
(5, SingleStringMathTex('^2'))
(6, SingleStringMathTex('='))
(7, SingleStringMathTex('{x}'))
(8, SingleStringMathTex('^2'))
(9, SingleStringMathTex('+'))
(10, SingleStringMathTex('2'))
(11, SingleStringMathTex('{x}'))
(12, SingleStringMathTex('{y}'))
(13, SingleStringMathTex(' + '))
(14, SingleStringMathTex('{y}'))
(15, SingleStringMathTex('^2'))
[6]


eqn3:
(0, SingleStringMathTex('4')) (1, SingleStringMathTex('{x}')) (2, SingleStringMathTex('{y}')) (3, SingleStringMathTex('=')) (4, SingleStringMathTex('\\big(')) (5, SingleStringMathTex('{x}')) (6, SingleStringMathTex('+')) (7, SingleStringMathTex('{y}')) (8, SingleStringMathTex('\\big)')) (9, SingleStringMathTex('^2')) (10, SingleStringMathTex('-')) (11, SingleStringMathTex('\\big(')) (12, SingleStringMathTex('{x}')) (13, SingleStringMathTex('-')) (14, SingleStringMathTex('{y}')) (15, SingleStringMathTex('\\big)')) (16, SingleStringMathTex('^2'))
eqn3a:
(0, SingleStringMathTex('\\big(')) (1, SingleStringMathTex('{x}')) (2, SingleStringMathTex('-')) (3, SingleStringMathTex('{y}')) (4, SingleStringMathTex('\\big)')) (5, SingleStringMathTex('^2')) (6, SingleStringMathTex('+')) (7, SingleStringMathTex('4')) (8, SingleStringMathTex('{x}')) (9, SingleStringMathTex('{y}')) (10, SingleStringMathTex('=')) (11, SingleStringMathTex('\\big(')) (12, SingleStringMathTex('{x}')) (13, SingleString

(0, SingleStringMathTex(' a '))
(1, SingleStringMathTex('^2'))
(2, SingleStringMathTex('+'))
(3, SingleStringMathTex(' b '))
(4, SingleStringMathTex('^2'))
(5, SingleStringMathTex('='))
(6, SingleStringMathTex(' c '))
(7, SingleStringMathTex('^2'))
(8, SingleStringMathTex('\\text{ has infinitely many integer solutions!}'))


In [23]:
print(UL)

[-1.  1.  0.]


In [166]:
%%manim -ql -v WARNING BinomialSquare

def crosshatch_rectangle(rect, spacing=0.15, angle=75*DEGREES, color=WHITE,lwidth=1):
    """
    Create crosshatch lines clipped to a Rectangle or Square,
    without using boolean operations.
    """

    # Rectangle corner coordinates (in world space)
    my_UL = rect.get_corner(UL)
    my_UR = rect.get_corner(UR)
    my_DL = rect.get_corner(DL)
    my_DR = rect.get_corner(DR)

    # Convert rectangle boundary to a list of segment endpoints
    edges = [
        (my_UL, my_UR),  # top
        (my_UR, my_DR),  # right
        (my_DL, my_DR),  # bottom
        (my_DL, my_UL),  # left
    ]

    # Determine a sufficiently large diagonal span
    width = rect.width
    height = rect.height
    diag = width + height

    # Number of diagonals needed
    n = int(diag / spacing) + 4

    hatch = VGroup()

    # Create and clip lines
    for i in range(-n, n+1):
        # Base line (long)
        p1 = LEFT*diag
        p2 = RIGHT*diag 

        # Rotate & shift
        R = rotation_matrix(angle, OUT)[:3,:3]
        p1 = R @ p1
        p2 = R @ p2

        # Shift the diagonal line by i steps upward
        shift_vec = (i +0.5) * spacing * (R@UP)
        p1 = p1 + shift_vec
        p2 = p2 + shift_vec

        p1 += rect.get_center()
        p2 += rect.get_center()

        # ------------------------------------------------------------------
        # Compute intersections between this diagonal line and the rectangle
        # ------------------------------------------------------------------
        pts = []
        for (a, b) in edges:
            inter = line_intersection(p1, p2, a, b)
            if inter is not None:
                # Check if intersection lies within rectangle edge segment
                if point_on_segment(inter, a, b):
                    # Check if intersection lies on (infinite) diagonal line segment
                    pts.append(inter)

        # We keep segments only if we have exactly 2 intersection points
        if len(pts) == 2:
            seg = Line(pts[0], pts[1], color=color, stroke_width=lwidth)
            seg.set_fill(color=color, opacity=0.1)
            hatch.add(seg)

    return hatch


# -------------------------------------------------------
# Helper: line intersection
# -------------------------------------------------------
def line_intersection(p1, p2, p3, p4):
    """
    Intersection of line segment p1→p2 with line segment p3→p4.
    Returns None or a numpy array.
    """

    x1, y1, _ = p1
    x2, y2, _ = p2
    x3, y3, _ = p3
    x4, y4, _ = p4

    denom = (x1-x2)*(y3-y4) - (y1-y2)*(x3-x4)
    if abs(denom) < 1e-8:
        return None  # parallel lines

    px = ((x1*y2 - y1*x2)*(x3 - x4) - (x1-x2)*(x3*y4 - y3*x4)) / denom
    py = ((x1*y2 - y1*x2)*(y3 - y4) - (y1-y2)*(x3*y4 - y3*x4)) / denom
    return np.array([px, py, 0])


# -------------------------------------------------------
# Helper: check if point lies on the interior of segment AB
# -------------------------------------------------------
def point_on_segment(p, a, b):
    return (
        np.allclose(np.linalg.norm(np.cross(b-a, p-a)), 0, atol=1e-6)
        and np.dot(p-a, p-b) <= 0
    )


def create_black_box(math_tex, buffer=0.1):
    """
    Creates a black rounded rectangle behind a MathTex object.

    Parameters:
    - math_tex: MathTex object to surround
    - buffer: padding around the MathTex

    Returns:
    - A VGroup containing the rectangle (ready to add behind)
    """
    rect = SurroundingRectangle(
        math_tex,
        color=BLACK,
        fill_opacity=1,
        buff=buffer,
        corner_radius=0.0
    )
    return rect


class BinomialSquare(Scene):
    def pause(self):
        self.wait(1)
    def construct(self):
        # lengths (manim units)
        x = 2
        y = 1.2

        # Outer big square
        big_square = Square(side_length=x+y, color=WHITE).shift(DOWN*0.3)

        # Regions
        # Top-left x^2
        tl = Rectangle(width=x, height=x).move_to(
            big_square.get_corner(UL) + DOWN*x/2 + RIGHT*x/2
            
        )
        #tl.set_fill(color=x_color, opacity=0.3)
        # Top-right xy
        tr = Rectangle(width=y, height=x).move_to(
            big_square.get_corner(UR) + DOWN*x/2 + LEFT*y/2
        )

        # Bottom-left xy
        bl = Rectangle(width=x, height=y).move_to(
            big_square.get_corner(DL) + UP*y/2 + RIGHT*x/2
        )

        # Bottom-right y^2
        br = Rectangle(width=y, height=y).move_to(
            big_square.get_corner(DR) + UP*y/2 + LEFT*y/2
        )

        # Labels for areas
        label_x2 = MathTex("{x}^2",tex_to_color_map=t2cD).move_to(tl.get_center())
        label_xy1 = MathTex("{x}{y}",tex_to_color_map=t2cD).move_to(tr.get_center())
        label_xy2 = MathTex("{x}{y}",tex_to_color_map=t2cD).move_to(bl.get_center())
        label_y2 = MathTex("{y}^2",tex_to_color_map=t2cD).move_to(br.get_center())

        # Side labels
        label_x_top = MathTex("{x}",tex_to_color_map=t2cD).next_to(
            big_square.get_top(), UP, buff=0.15
        ).shift(LEFT * y/2)

        label_y_top = MathTex("{y}",tex_to_color_map=t2cD).next_to(
            big_square.get_top(), UP, buff=0.15
        ).shift(RIGHT * x/2)

        label_x_left = MathTex("{x}",tex_to_color_map=t2cD).next_to(
            big_square.get_left(), LEFT, buff=0.15
        ).shift(UP * y/2)

        label_y_left = MathTex("{y}",tex_to_color_map=t2cD).next_to(
            big_square.get_left(), LEFT, buff=0.15
        ).shift(DOWN * x/2)

        self.add(label_x_top,label_y_top,label_x_left,label_y_left)
        self.play(Create(big_square))
        self.play(*[Create(mob) for mob in [tl,tr,bl,br]])


        #self.add(big_square,tl,tr,bl,br,label_x_top,label_y_top,label_x_left,label_y_left)

        x_spacing = 0.2
        y_spacing = 0.1
        parts = [(tl,label_x2,"x","x"),(br,label_y2,"y","y"),(bl,label_xy2,"x","y"),(tr,label_xy1,"y","x")]
        for rect, label, vert_type, horz_type in parts:
            vert_color = x_color if vert_type == "x" else y_color
            horz_color = x_color if horz_type == "x" else y_color
            vert_s = x_spacing if vert_type == "x" else y_spacing
            horz_s = x_spacing if horz_type == "x" else y_spacing

            vert_lwidth = 3 if vert_type == "x" else 1
            horz_lwidth = 3 if horz_type == "x" else 1
            
            v_hatch = crosshatch_rectangle(rect, color=vert_color,lwidth=vert_lwidth,spacing=vert_s)
            h_hatch = crosshatch_rectangle(rect, angle=0*DEGREES, color=horz_color,lwidth=horz_lwidth,spacing=horz_s)
            black_box = create_black_box(label)
            label.z_index = black_box.z_index+1
            self.play(GrowFromEdge(v_hatch,edge=UP))
            self.play(GrowFromEdge(h_hatch,edge=LEFT))
            self.play(GrowFromCenter(VGroup(black_box,label)))
            self.pause()
             
             

        

        # self.play(Create(big_square))
        # self.play(
        #     Create(tl), Create(tr),
        #     Create(bl), Create(br)
        # )
        # self.play(
        #     Write(label_x2), Write(label_xy1),
        #     Write(label_xy2), Write(label_y2)
        # )
        # self.play(
        #     Write(label_x_top), Write(label_y_top),
        #     Write(label_x_left), Write(label_y_left)
        # )

        #self.wait()


Manim Community v0.19.0